In [ ]:
!pip install datasets # Installe la librairie Hugging Face Datasets pour charger des jeux de données standardisés.

In [ ]:
!pip install transformers -U # Installe ou met à jour Transformers pour télécharger modèles et tokenizers.
!pip install accelerate -U # Installe Accelerate pour gérer CPU/GPU/TPU et paralléliser l’entraînement.

In [ ]:
!pip install trl # Installe TRL (Transformer Reinforcement Learning). Ici utilisé pour l’entraînement supervisé simplifié (SFT).

In [ ]:
import torch # Charge PyTorch.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Sélectionne le GPU si présent, sinon le CPU. Servira à placer tenseurs et modèles.

In [ ]:
from datasets import load_dataset # Importe la fonction de chargement HF.
DATASET_NAME = "mlabonne/guanaco-llama2-1k" # Nom du dataset à récupérer depuis Hugging Face Hub.
#Le dataset a un format de type chat/instruction.

dataset = load_dataset(DATASET_NAME) # Télécharge et met en cache local le dataset. Retourne un DatasetDict (splits).

In [ ]:
print(dataset) # Affiche la structure des splits disponibles.

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1000
    })
})


In [ ]:
training_dataset = dataset['train'] # Sélectionne le split d’entraînement.
print(training_dataset) # Affiche nombre d’exemples et les colonnes.

Dataset({
    features: ['text'],
    num_rows: 1000
})


In [ ]:
training_dataset[0] # Indexe le premier exemple. En notebook, affiche un dict.

{'text': '<s>[INST] Me gradué hace poco de la carrera de medicina ¿Me podrías aconsejar para conseguir rápidamente un puesto de trabajo? [/INST] Esto vale tanto para médicos como para cualquier otra profesión tras finalizar los estudios aniversarios y mi consejo sería preguntar a cuántas personas haya conocido mejor. En este caso, mi primera opción sería hablar con otros profesionales médicos, echar currículos en hospitales y cualquier centro de salud. En paralelo, trabajaría por mejorar mi marca personal como médico mediante un blog o formas digitales de comunicación como los vídeos. Y, para mejorar las posibilidades de encontrar trabajo, también participaría en congresos y encuentros para conseguir más contactos. Y, además de todo lo anterior, seguiría estudiando para presentarme a las oposiciones y ejercer la medicina en el sector público de mi país. </s>'}

In [ ]:
training_dataset[11] # Affiche l’exemple d’index 11.

{'text': '<s>[INST] write me a 1000 words essay about deez nuts. [/INST] The Deez Nuts meme first gained popularity in 2015 on the social media platform Vine. The video featured a young man named Rodney Bullard, who recorded himself asking people if they had heard of a particular rapper. When they responded that they had not, he would respond with the phrase "Deez Nuts" and film their reactions. The video quickly went viral, and the phrase became a popular meme. \n\nSince then, Deez Nuts has been used in a variety of contexts to interrupt conversations, derail discussions, or simply add humor to a situation. It has been used in internet memes, in popular music, and even in politics. In the 2016 US presidential election, a 15-year-old boy named Brady Olson registered as an independent candidate under the name Deez Nuts. He gained some traction in the polls and even made appearances on national news programs.\n\nThe Deez Nuts meme has had a significant impact on popular culture. It has b

In [ ]:
training_dataset[7] # Affiche l’exemple d’index 7.
# Objectif de ces trois dernières lignes: inspection rapide du format.

{'text': '<s>[INST] Ich möchte für ein Handy-Game den Flug von Papierfliegern simulieren. Was für Formeln können mir helfen um das möglichst realistisch zu tun? [/INST] Um den Flug von Papierfliegern in einem Spiel realistisch zu simulieren, gibt es einige wichtige Formeln, die Sie berücksichtigen sollten:\n\nLift-Kraft: Der Lift, der einen Papierflieger in der Luft hält, wird durch die Bewegung der Luft über die Oberfläche des Flügels erzeugt. Die Formel zur Berechnung der Lift-Kraft lautet:\n\nL = 0.5 * rho * v^2 * A * Cl\n\nL = Lift-Kraft\nrho = Luftdichte\nv = Geschwindigkeit des Flugzeugs\nA = Flügeloberfläche\nCl = Auftriebsbeiwert des Flügels\n\nLuftwiderstand: Der Luftwiderstand ist die Kraft, die gegen den Flug eines Papierfliegers wirkt und durch die Bewegung der Luft entsteht, wenn sie um das Flugzeug herumströmt. Die Formel zur Berechnung des Luftwiderstands lautet:\n\nD = 0.5 * rho * v^2 * A * Cd\n\nD = Luftwiderstand\nrho = Luftdichte\nv = Geschwindigkeit des Flugzeugs\nA

In [ ]:
MODEL_NAME = "distilgpt2" # Choisit DistilGPT-2, version légère de GPT-2.
import transformers # Charge le package Transformers.
from transformers import AutoModelForCausalLM # API générique pour charger un modèle de langage causal (next-token).
from transformers import AutoTokenizer # API générique pour charger le tokenizer compatible.

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map = "auto") # Télécharge le modèle pré-entraîné et le place automatiquement sur CPU/GPU(s) selon la RAM VRAM disponible.
model.config.use_cache = True # Active le cache de clés/valeurs pour accélérer la génération auto-rgressive.
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True) # Télécharge le tokenizer lié au modèle. trust_remote_code=True permet du code de tokenizer custom si besoin.
# Below, we use padding to get all tensors with same length
tokenizer.pad_token = tokenizer.eos_token # Définit le token de padding égal au token EOS. GPT-2 n’a pas de token PAD natif, on réutilise EOS (end of sentence).
tokenizer.padding_side = 'right' # Pad à droite des séquences. Important pour cohérence forme [tokens, PAD, PAD].
tokenizer.pad_token_id = tokenizer.eos_token_id # Synchronise l’ID du token de pad avec celui d’EOS.

generation_configuration = model.generation_config # Récupère l’objet de configuration de génération attaché au modèle.
generation_configuration.pad_token_id = tokenizer.eos_token_id # Assure cohérence entre modèle et tokenizer pour le padding.
generation_configuration.eos_token_id = tokenizer.eos_token_id # Spécifie l’ID du token de fin de séquence.
generation_configuration.max_new_tokens = 1024 # Limite le nombre de nouveaux tokens générés. Note: le contexte total GPT-2 est 1024, demander 1024 new tokens peut tronquer l’input si long.

# Below variables depend on how we sample our model
generation_configuration.temperature = 0.7 # Ajuste l’entropie de l’échantillonnage. Plus bas = plus déterministe.
generation_configuration.top_p = 0.9 # Nucleus sampling: ne tire que dans le plus petit ensemble de tokens dont la probabilité cumulée ≥ 0.9.
generation_configuration.top_k = 20 # Tronque le vocabulaire aux 20 tokens les plus probables avant échantillonnage.
generation_configuration.do_sample = True # Enable sampling for generation

In [ ]:
def generate(prompt): # Déclare une fonction utilitaire.

  encoded = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt").to(device) # Tokenise le prompt en IDs, ajoute tokens spéciaux si définis, retourne un tenseur PyTorch et l’envoie sur device.
  out = model.generate(input_ids=encoded, repetition_penalty=2.0, do_sample=True) # Lance la génération auto-rressive avec pénalité de répétition (pénalise la ré-utilisation abusive de n-grams) et avec échantillonnage stochastique activé.
  string_decoded = tokenizer.decode(out[0].tolist(), clean_up_tokenization_spaces=True) # Convertit les IDs générés en texte brut.
  print(string_decoded) # Affiche le texte produit.

In [ ]:
generate('this is') # Exemple d’appel. Produit une complétion.

this is the and that both both both both two as in which
 ()) if than, to or any oror:. - -- a not know they we of<|endoftext|>


In [ ]:
generate('how are you') # Deuxième exemple.

how are you?›
The answer to this question is no, and I think the answers in that article will be interesting. In fact it's not even remotely important for me if they're anything like my own or those of a group who have come out with similar opinions about myself:<|endoftext|>


In [ ]:
from trl import SFTConfig, SFTTrainer # Importe la config et l’entraîneur SFT (Supervised Fine-Tuning).
from transformers import TrainingArguments # Importe la classe d’arguments HF. Ici on n’utilise que SFTConfig, mais import ok.

# set up training arguments
training_args = SFTConfig( #Crée une config d’entraînement supervisé.
      gradient_accumulation_steps=1, # Pas d’accumulation de gradient. Un update par batch.
      num_train_epochs=1, # Une époque sur le split train.
      learning_rate=2e-4, # LR initiale. Assez élevée pour un petit modèle.
      fp16=True, # Active Half-Precision si GPU le permet. Gagne en vitesse/mémoire.
      output_dir="logs", # Dossier de sorties: checkpoints, logs.
      lr_scheduler_type="cosine", # Plan de décroissance du LR cosinus.
      warmup_ratio=0.05, # 5% d’étapes en warmup pour stabiliser les gradients.
      group_by_length=True, # Regroupe par longueur pour réduire padding et accélérer.
      max_length=512, # Tronque les entrées à 512 tokens pour l’entraînement.
      # neftune_noise_alpha=5, # Add noise to the embeddings (NEFTune) to improve robustness.
      do_sample=True, # Explicitly enable sampling in the training config
) # Fin de la Config

# Fine Tuning Dataset
# Learning Rate is critical to Gradient Descent
# A dynamic learning rate is best (dynamic alpha)
# Towards the end of training -> small learning rate

# batch_size cannot be very large -> GPU memory limitation -> do an iteration of training, calculate gradients, but don't update the weights
# do another iteration, calculate gradients, add them to the previous iteration's gradients
# and then update the weights

#initialize the SFTTrainer
trainer = SFTTrainer(model=model, train_dataset=training_dataset, processing_class=tokenizer, args=training_args)
# Construit un entraîneur SFT:
# model: DistilGPT-2 chargé
# train_dataset: split train du dataset
# processing_class=tokenizer: objet utilisé pour tokeniser et formater les batchs
# args: hyperparamètres ci-dessus
# Remarque: pour un dataset chat, il faut souvent préciser quelles colonnes contiennent prompt/réponse, ou fournir un template de formatage.
# Ici, TRL infère si les colonnes correspondent au schéma attendu du dataset choisi.

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [ ]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.


Step,Training Loss
10,3.309500
20,3.322100
30,3.655800
40,3.335300
50,3.265300
60,3.435100
70,3.296600
80,3.141700
90,3.298700
100,3.156400


TrainOutput(global_step=125, training_loss=3.326212585449219, metrics={'train_runtime': 46.4003, 'train_samples_per_second': 21.552, 'train_steps_per_second': 2.694, 'total_flos': 94701699661824.0, 'train_loss': 3.326212585449219, 'entropy': 3.438341569900513, 'num_tokens': 363618.0, 'mean_token_accuracy': 0.37988496422767637, 'epoch': 1.0})

In [ ]:
import wandb

# Initialize wandb
wandb.init(project="your_project_name", name="your_run_name") # Replace with your desired project and run names

train/entropy,▄▅▆▄▄▄▃▆▂▃▅▅█▅▅▅▃▅▃▄▃▃▃▂▃▁▃▂▂▃▅█▅▅▆▃▅▃▄▆
train/epoch,▁▂▂▃▃▅▅▆▆▇▁▂▂▃▃▅▅▆▇█▂▂▃▃▄▅▆▆▇█▂▂▃▃▄▅▆▆▇█
train/global_step,▁▂▂▃▃▅▅▆▆▇▁▂▂▃▃▅▅▆▆▇▁▂▂▃▃▅▆▆▇█▂▂▃▃▄▅▆▆▇█
train/grad_norm,▃▄█▃▃▂▂▅▂▂▄ ▇▂▂▂▁▃▁▁▁▂▆▂▂▃▂▃▁▂▄ █▂▂▂▁▃▁▃
train/learning_rate,██▇▇▆▄▃▂▂▁██▇▇▆▄▃▂▂▁██▇▇▆▄▃▂▂▁██▇▇▆▄▃▂▂▁
train/loss,▃▅▆▄▅▄▃▇▃▃▅▅█▅▅▅▃▅▄▄▁▃▃▃▃▃▁▄▁▂▅▅█▅▅▅▃▅▄▆
train/mean_token_accuracy,▃▄▂▄▅▄▇▃▄▅▃▄▁▂▅▃▅▄▄▅▆▇▆▅▅▅█▅▇▇▄▁▂▅▃▅▄▄▅▄
train/num_tokens,▂▂▂▃▃▄▄▄▄▅▁▁▁▂▂▃▃▃▄▄▅▅▆▆▆▇▇▇██▁▁▂▂▂▃▃▄▄▄
total_flos,94701699661824.0
train/entropy,3.43834
train/epoch,1


In [ ]:
generate('how are you')

how are you?
 (). for that<|endoftext|>


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
